# ***Titulo***

# **Predicción de la Calidad del Agua en México Mediante Algoritmos de Aprendizaje Automático**

***`Vizuet_CF ⛺`***



## **Descripción**

Se utilizaron los datos de la calidad del agua (CA), provenientes de la Comisión Nacional del Agua (CONAGUA., 2021), que consta de 2,500 sitios de monitoreo, de los cuales 788 corresponden al análisis de aguas superficiales que considera 8 indicadores.

### **Indicadores**

1. Demanda Bioquímica de Oxígeno a cinco días (DBO5)
2. Demanda Química de Oxígeno (DQO)
3. Sólidos Suspendidos Totales (SST)
4. Coliformes Fecales (CF)
5. *Escherichia coli* (E_COLI)
6. Enterococos (ENTEROC)
7. Porcentaje de Saturación de Oxigeno (OD%)
8. Toxicidad (TOX)

## **Semáforo**

La calidad del agua se determinó a través de un semáforo que considera 3 colores verde, amarillo y rojo, y se obtiene integrando los resultados de los 8 indicadores antes mencionados (CONAGUA., 2021).

## **Preprocesamiento de los datos**

Se eliminaron dos indicadores de la CA (ENTEROC y TOX); el data set utilizado está compuesto por 8 columnas.

**Nombre de las columnas**
1. DBO_mg/L
2. DQO_mg/L              
3. SST_mg/L              
4. COLI_FEC_NMP_100mL    
5. E_COLI_NMP_100mL      
6. OD_PORC               
7. SEMAFORO               
8. NUM_SEMAFORO

Los colores del semáforo de la calidad del agua se ordenaron alfabéticamente y se les asignó un numero (1, 2 y 3); posteriormente se almacenaron en la columna 8 (NUM_SEMAFORO).

**Numero por color**

- Amarillo - 1
- Rojo - 2
- Verde - 3

## **Entrenamiento**

Para el entrenamiento del modelo se tomaron en cuenta 6 indicadores (DBO5, DQO, SST, CF, E_COLIi y OD%) para predecir la CA de acuerdo al numero asignado para cada color del semaforo.

## Randomized

### Importamos las librerías

In [1]:
# librerias
import pandas as pd
import numpy as np
# modulos
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

import gspread
from oauth2client.service_account import ServiceAccountCredentials

import warnings
warnings.filterwarnings("ignore")

# **Algoritmo**

### Creamos la conexión con Google o subimos el archivo desde nuestra computadora

In [2]:
if __name__ == "__main__":

    # creamos la conexión con google

    from google.colab import drive
    drive.mount('/content/drive')    # nobre de la conexion

    # o subimos un archivo (si es necesario)

    # from google.colab import files
    # Uploaded = files.upload()

Mounted at /content/drive


### Creamos una variable con el nuevo archivo

In [7]:
    ## leemos los archivos

    ### leemos el archivo y especificamos la hoja que usaremos como df
    dataset = pd.read_excel('/content/drive/MyDrive/UAM/Calidad del agua /ica_sup_1.xlsx')


    ## tratamiento de los valores atipicos y nulos

    ### eliminar los valores nulos
    ch_dbo_1_0 = dataset.dropna()

    ### cambia los valores atipicos por 0
    ch_dbo_1_0_0 = ch_dbo_1_0.replace({'<30':30, '<3':3, '<2':2, '<10':10})

    ### tamaño de los datos
    print(dataset.shape)
    print(ch_dbo_1_0.shape)
    # ### tipos de datos
    print(ch_dbo_1_0_0.dtypes)
    ### descripción del df
    print(ch_dbo_1_0_0.describe())
    ### eliminamos las columnas que no usaremos
    ch_dbo_1_0_1 = ch_dbo_1_0_0.drop('SITIO', axis = 1)
    print(ch_dbo_1_0_1)

    # ch_dbo_1_0_1.to_csv('ind_ica.csv')

    # from google.colab import files
    # files.download('ind_ica.csv')

(4341, 9)
(2268, 9)
SITIO                  object
DBO_mg/L              float64
DQO_mg/L              float64
SST_mg/L              float64
COLI_FEC_NMP_100mL    float64
E_COLI_NMP_100mL      float64
OD_PORC               float64
SEMAFORO               object
NUM_SEMAFORO            int64
dtype: object
          DBO_mg/L     DQO_mg/L     SST_mg/L  COLI_FEC_NMP_100mL  \
count  2268.000000  2268.000000  2268.000000        2.268000e+03   
mean     22.481809    78.710625    50.311574        1.465129e+04   
std      90.955348   253.712341   149.143927        9.360976e+04   
min       2.000000    10.000000    10.000000        3.000000e+00   
25%       2.000000    15.255000    11.000000        1.100000e+03   
50%       2.972500    26.975000    24.000000        2.812500e+03   
75%      10.866250    57.000000    52.000000        2.400000e+04   
max    2550.960000  7632.000000  4276.000000        2.419600e+06   

       E_COLI_NMP_100mL      OD_PORC  NUM_SEMAFORO  
count      2.268000e+03  2268.

## Entrenamiento

In [8]:
    # es importante que exista una correlacion  entre las features y la variable que queremos predecir y no es necesarios que los features la tengan

    X = ch_dbo_1_0_1.drop(['SEMAFORO', 'NUM_SEMAFORO'], axis=1)
    y = ch_dbo_1_0_1[['NUM_SEMAFORO']]
    reg = RandomForestRegressor()

    parameters = {
        'n_estimators': range(4, 16), # cuantos arboles compondran mi arbol
        'criterion': ['friedman_mse', 'absolute_error'],
        'max_depth': range(2, 11)
    }

    ### son 10 iteracion del optimizador. Toma 10 combinaciones al azar del diccionario, cv = 3, parte en 3 parte el set de datos que le pasemos, para hacer Cross validation
    rand_est = RandomizedSearchCV(reg, parameters, n_iter = 5, cv = 3, scoring = 'neg_mean_absolute_error').fit(X, y)

## Predicciones y Pruebas

In [17]:
    ## mejores estimadores y parametros

    print('='*64)
    print("Mejores estimadores")
    print('-'*64)
    print(rand_est.best_estimator_)
    print('='*64)
    print("Mejores parametros")
    print('-'*64)
    print(rand_est.best_params_)
    print('='*64)

    ## pruebas

    print('Pruebas')
    print('-'*64)
    y_hat = rand_est.predict(X.loc[[13]])

    ## datos reales

    print(f'Real:    {y.loc[13]}')
    print('='*64)

    ## predicciones

    print(f'Predict: {y_hat[0]}')

Mejores estimadores
----------------------------------------------------------------
RandomForestRegressor(criterion='absolute_error', max_depth=3, n_estimators=4)
Mejores parametros
----------------------------------------------------------------
{'n_estimators': 4, 'max_depth': 3, 'criterion': 'absolute_error'}
Pruebas
----------------------------------------------------------------
Real:    NUM_SEMAFORO    1
Name: 13, dtype: int64
Predict: 1.0


In [19]:
    X_test = np.array([5.58,39.92,14.175,5172.0,925.0,52.8])
    y_pre = rand_est.predict(X_test.reshape(1, -1))

    print(f'''El ICA  es:
    Redondeo: {np.round(y_pre[0], 0)}
    Sin redondear: {y_pre}''')

El ICA  es:
Redondeo: 1.0
Sin redondear: [1.]
